*Recordando: Dentro del archivo PDB tendrémos la estructura de las proteínas. 


ahora...

Podemos acceder a los átomos

In [ ]:
#Para acceder a los átomos podemos iterar en el objeto residue
residue = chain[22]
for atom in residue:
  print(atom)
#También podemos acceder a ellos como diccionario o lista
atom = residue['CA']
atom = residue.child_list[1]

#También podemos ver la información de su posición 3D
at_1.coord

In [ ]:
#Para los Átomos

residue = chain[22]
print(residue.get_resname())

for atom in residue:
  print(atom)

atom_ca = residue['CB']

print(atom_ca)
#coordenadas del átomo
print(atom_ca.coord)
#elemento que representa
print(atom_ca.element)
#id
print(atom_ca.id)


Otros métodos para obtener a los "child"


In [ ]:
chainss = model.get_chains()
prnt(*chainss)

residuess = chain.get_residues()
print(*residuess)

atomss = residue.get_atoms()
for atom in atomss:
  print(atom)
  

## Ejercicio 3
Imprime los elementos y los ids de los átomos del primer residuo de cisteína que tenemos en la lista cys_A para conocer el id del elemento s

In [ ]:
# Retomando la lista de cisteinas que ya hicimos
cisteinas = []

for modelo in obj_struc:
  for chain in modelo:
    # Tipo de cadena - .id
    if chain.id =='A':
      for residuo in chain:
        # Aminoácido - .get_resname() 
        if residuo.get_resname() == 'CYS':
          cisteinas.append(residuo)

print(len(cisteinas))

#Ejercicio 3
for atom in cisteinas[0]:
  print(atom.element, atom.id)


También podemos ver la distancia entre átomos restando:

In [ ]:
atomo_cb = residue['CB']
atomo_ca = residue['CA']

atomo_cb - atomo_ca

## Ejercicio 4
Calcular la distancia de todos los átomos de s para encontrar los que estan cerca.

calcular las distancias de los átomos s de las cisteínas de la lista cys_A.
Guardar los pares de cisteínas de aquellos que tengan una distancia menor a 8 A.

In [ ]:
for cist_1 in cisteinas:
  for cist_2 in cisteinas:
    #Condición para no comparase consigo misma
    if cist_1 != cist_2:

      if(cist_1['SG'] - cist_2['SG'])  < 8:
        #coordenadas de los átomos
        print(cist_1['SG'].coord, cist_2['SG'].coord)
        #pares de ID de los residuos
        print(cist_1.id, cist_2.id)


## Otros formatos que se pueden trabajar:
* Fasta
* Fastq
* SFF
* GenBank
* PDB
* PQR
* CIF
* Outputs de BLAST
* Alineamientos

# BASES DE DATOS

Inicios de las bases de datos

(1965) Atlas of Protein Sequence and Structure - Secuencias (+/- 70 proteínas con alineamientos y referencias de las secuencias)
* Busqueda manual en publicaciónes
* Verificación de secuencias

Diferentes propuestas para bases de datos:
* Propuesta Dayhoff (copyright)
* Propuesta Goad (conocimiento público)

--- 

EMBL (1980)

1981- Primer repositorio central de secuencias open acces

Los Álamos (precursor de genebank):
coperación internacional  EMBL y DDBJ

NCBI 1990-1999

**COLABORACIÓN INTERNACIONAL**
INSD (DDBJ - NCBI - ENA/EBI)

*principios FAIR (estandarización)


# Entrez
E-utilities de NCBI

Interacción entre bases de datos desde la web 

**Para trbajos muy grandes podemos automatizar**

---
URL -> E-utilities -> fasta,xml,...

---
Existen diferentes posibles pipelines a seguir

**NCBI tiene limitaciones:**

Para usar e-utilities 

* Máx 3 request x segundo (upgrade a 10 con llave API)
* Anexar correo !

### Entrez.einfo y Entrez.read

In [ ]:
### Einfo ###

from Bio import Entrez
#mejor visualización de diccionrios
from pprint import pprint 
#Correo 
Entrez.emali = "mmayen@lcg.unam.mx"
#Handle
handle = Entrez.einfo()
record = Entrez.read(handle)
handle.close()
#revisar qué hay en einfo
print(result)


In [ ]:
### Entrez.read ###

handle = Entrez.einfo()
record = Entrez.read(handle)
#obtenemos diccionario (llave "Dblist")
print(record['Dblist'][0:3])  #Imprime primeras 3 bases de daos
handle.close()  #cerrar handle

In [ ]:
# Para obtener información de genome

from Bio import Entrez
Entrez.email = "mmayen@lcg.unam.mx"

# db = base de datos de interés
handle = Entrez.einfo(db="genome")  
record = Entrez.read(handle)
#Descripción
print(record['DbInfo']['Description'])
print(record['DbInfo']['FieldList'])
#URL accesado para la búsqueda
print(handle.url)


In [ ]:
#Si quiesiéramos conocer todos los campos a los que podemos accesar (pubmed)
# podemos imprimir todos 
for field in record['DbInfo']['FieldList']:
  print("%(Name)s, %(FullName)s, %(Description)s" %field)

## Ejercicio 1
Usando Entrez.einfo y Entrez.read imprimir la descripción de 2 campos de genome



In [ ]:
from Bio import Entrez
Entrez.email = "mmayen@lcg.unam.mx"

handle = Entrez.einfo(db="genome")  
record = Entrez.read(handle)

i =-1
for field in record['DbInfo']['FieldList']:
  i += 1
  if field['Namme'] == "ORGN":
    print(field['Name'], field['Description'])

#Accediendo a la posición 4 (organismo)
print(record['DbInfo']['FieldList'][3]['Description'])

### Entrez.esearch

Entrez.esearch(base de datos a buscar, término)


In [ ]:
#Buscar el término "biopython" en PubMed

handle = Entrez.esearch(db="pubmed", term="biopython")
record = Entrez.read(handle)
#checar cuántos resultados obtenemos con record["Count"]
record["Count"]
#cerrar handle
handle.close() 

## retmax
Es un parámetro dentro de la búsqueda que indica el máximo  de retrieves (default 20)

Podemos modificarlo según querramos (en este caso para obtener todos los resultados encontrados)

In [ ]:
#Obtenemos el tamaño de count
count = int(record["Count"])

handle = Entrez.esearch(db="pubmed", term="biopython", retmax=count)
record = Entrez.read(handle)
#cerrar handle
handle.close() 
#Ahora la longitud de IdList es de 35, igual que el tamaño de count
len(record["IdList"])  

## Ejercicio: buscando por autor


In [ ]:
#Usamos el parámetro field="AUTH"
handle = Entrez.esearch(db="pubmed", term="Valeria Mateo-Estrada", field="AUTH")
record = Entrez.read(handle)

#cerrar handle
handle.close()
#Obtenemos os IDs
record["IdList"] 

## Usando operadores booleanos
Podemos usarlos para realizar búsquedas con mayor especificidad


```
# Buscando el gen1 o el gen2 del organismo 1
Organismo1[Orgn] AND (gene1[Gene] OR gene2[Gene])
```



In [ ]:
#Ejemplo de búsqueda compleja
termino = (Aedes[Title] OR Aedes[All Fields]) AND ((RNA.Seq[Title] OR transcriptome[Title]) OR (transcriptome[Title] OR sequencing[Title]))